In [6]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [9]:
# Note 0 = business, 1 = entertainment, 2 = politics, 3 = sport, 4 = tech

def load_extract_data():

    classesFile = open('./bbc_folder/bbc.classes', 'r')
    mtxFile = open('./bbc_folder/bbc.mtx', 'r')

    numTerms = 9635

    # read the lines in these 2 files which do not contribute to the data (the headers)
    classesFile.readline(); classesFile.readline(); classesFile.readline(); classesFile.readline()
    mtxFile.readline(); 

    # load the classes file into a numpy array and split the data so that we have a label for each article
    array = np.loadtxt(classesFile, delimiter=" ").astype(int)
    (X, labels) = np.hsplit(array, 2)

    # read the mtx matrix as a csv
    df = pd.read_csv(mtxFile, delimiter=" ").astype(int)

    # define binary and frequency arrays for parts a) and b)
    totalTermsBinary = np.empty((X.shape[0], 9635))
    totalTermsFrequency = np.empty((X.shape[0], 9635))
    
    # for every article in which we have extracted the labels
    for i in X:
        index = i[0]+1
        # get all the rows associated with the article i (column 2 of mtx)
        articles = df.loc[df['2225'] == index]
        termsBinary = np.zeros((numTerms))
        termsFrequency = np.zeros((numTerms))

        # get the columns with the term and the frequency of that term for each article in rows
        foundTerms = articles[['9635', '286774']].to_numpy()

        for (j,k) in foundTerms:
            # If the term is in the article, assign the frequency/binary label to the corresponding term index
            termsFrequency[j-1] = k
            termsBinary[j-1] = 1

        # Add the extracted data to our final data tables
        totalTermsBinary[index-1] = termsBinary
        totalTermsFrequency[index-1] = termsFrequency
        
    classesFile.close()
    mtxFile.close()

    return totalTermsBinary, totalTermsFrequency, labels
 

    
    
    
    


In [10]:
X_binary, X_frequency, labels = load_extract_data()
print("-----------------------------------")
print(X_binary)
print(X_binary.shape)
print("-----------------------------------")
print(X_frequency)
print(X_frequency.shape)
print("-----------------------------------")
print(labels)
print(labels.shape)
print("-----------------------------------")

-----------------------------------
[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
(2225, 9635)
-----------------------------------
[[1. 5. 2. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
(2225, 9635)
-----------------------------------
[[0]
 [0]
 [0]
 ...
 [4]
 [4]
 [4]]
(2225, 1)
-----------------------------------


# Part a)

## Split Data For Naive Bayes

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X_binary, labels, test_size=0.30, train_size=0.70, random_state=20)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(1557, 9635)
(668, 9635)
(1557, 1)
(668, 1)


In [12]:
# Determine the initial probability of each class
def get_class_probabilities(labels):
    total = len(labels)
    probability_of_c0 = (total - np.count_nonzero(labels)) / total
    probability_of_c1 = np.count_nonzero(labels == 1) / total
    probability_of_c2 = np.count_nonzero(labels == 2) / total
    probability_of_c3 = np.count_nonzero(labels == 3) / total
    probability_of_c4 = np.count_nonzero(labels == 4) / total

    return probability_of_c0, probability_of_c1, probability_of_c2, probability_of_c3, probability_of_c4


In [13]:
probability_of_c0, probability_of_c1, probability_of_c2, probability_of_c3, probability_of_c4 = get_class_probabilities(Y_train)
print(probability_of_c0, probability_of_c1, probability_of_c2, probability_of_c3, probability_of_c4)

0.22286448298008993 0.16955684007707128 0.19524727039177905 0.23249839434810532 0.1798330122029544


In [14]:
# Get the probability that if the term is in the article that it is the wanted label
def get_probability(X_train, labels, term, wanted_label, total_counts):

    count = 0
    index = 0
    for row in X_train:
        # if we have the term is in the article type we want increment the count
        if row[term] == 1 and labels[index] == wanted_label:
            count+=1
        index += 1
    prob = (count + 1) / (total_counts[wanted_label] + 2)
    return prob



In [15]:
# Get the probability that if the term is not in the article it is label wanted_label
def get_not_probability(X_train, labels, term, wanted_label, total_counts):
    count = 0
    index = 0
    for row in X_train:
        # If the term is not in our article type increment the count
        if row[term] == 0 and labels[index] == wanted_label:
            count+=1
        index += 1
    prob = (count + 1) / (total_counts[wanted_label] + 2)
    return prob

In [16]:
def naive_bayes_train(X_train, Y_train):

    probTable = np.empty((9635, 10))

    total_counts = np.bincount(Y_train[:,0])
    for i in range(9635):
        # probability that term i is 1 and leads to label 0
        probTable[i][0] = get_probability(X_train, Y_train, i, 0, total_counts)
        # Probability that term i not 1 and leads to label 0
        probTable[i][1] = 1 - probTable[i][0]

        # probability that term i is 1 and leads to label 1
        probTable[i][2] = get_probability(X_train, Y_train, i, 1, total_counts)
        # Probability that term i not 1 and leads to label 1
        probTable[i][3] = 1- probTable[i][2]
        
        # probability that term i is 1 and leads to label 2
        probTable[i][4] = get_probability(X_train, Y_train, i, 2, total_counts)
        # Probability that term i not 1 and leads to label 2
        probTable[i][5] = 1-probTable[i][4]

        # probability that term i is 1 and leads to label 3
        probTable[i][6] =  get_probability(X_train, Y_train, i, 3, total_counts)
        # Probability that term i not 1 and leads to label 3
        probTable[i][7] = 1 - probTable[i][6]

        # probability that term i is 1 and leads to label 4
        probTable[i][8] = get_probability(X_train, Y_train, i, 4, total_counts)
        # Probability that term i not 1 and leads to label 4
        probTable[i][9] = 1 - probTable[i][8]

    return probTable
        


    

In [17]:
probabilities = naive_bayes_train(X_train, Y_train)
print(probabilities)

[[0.26074499 0.73925501 0.22180451 ... 0.66208791 0.30141844 0.69858156]
 [0.30659026 0.69340974 0.18796992 ... 0.93956044 0.17730496 0.82269504]
 [0.17191977 0.82808023 0.04511278 ... 0.96428571 0.06382979 0.93617021]
 ...
 [0.00286533 0.99713467 0.0037594  ... 0.99725275 0.0141844  0.9858156 ]
 [0.00286533 0.99713467 0.0037594  ... 0.99725275 0.0141844  0.9858156 ]
 [0.00286533 0.99713467 0.0037594  ... 0.99725275 0.0106383  0.9893617 ]]


## Predict Naive Bayes

In [18]:
def naive_bayes_predict(X_test, probabilities, Y_train):
    prediction = np.empty((X_test.shape[0]))

    init_prob0, init_prob1, init_prob2, init_prob3, init_prob4 = get_class_probabilities(Y_train)
    init_prob0, init_prob1, init_prob2, init_prob3, init_prob4 = np.log(init_prob0), np.log(init_prob1), np.log(init_prob2), np.log(init_prob3), np.log(init_prob4)
    
    index = 0
    for row in X_test:
        prob0 = init_prob0
        prob1 = init_prob1
        prob2 = init_prob2
        prob3 = init_prob3
        prob4 = init_prob4

        # find all terms that are not present in article
        zeros = np.where(row == 0)
        # find all terms that are present in article
        ones = np.where(row == 1)


        # get the total log probability where probabilities[ones][;,i] is all the rows where the term is 1 and the 
        # column i which is the probability that the term is in article i/2
        prob0 += np.sum(np.log(probabilities[ones][:,0])) + np.sum(np.log(probabilities[zeros][:,1]))
        prob1 += np.sum(np.log(probabilities[ones][:,2])) + np.sum(np.log(probabilities[zeros][:,3]))
        prob2 += np.sum(np.log(probabilities[ones][:,4])) + np.sum(np.log(probabilities[zeros][:,5]))
        prob3 += np.sum(np.log(probabilities[ones][:,6])) + np.sum(np.log(probabilities[zeros][:,7]))
        prob4 += np.sum(np.log(probabilities[ones][:,8])) + np.sum(np.log(probabilities[zeros][:,9]))

        
        # Note for highest probability we just find the highest log and assign that class with the highest probability
        maximum = max(prob0, prob1, prob2, prob3, prob4)

        if prob0 == maximum:
            prediction[index] = 0
        elif prob1 == maximum:
            prediction[index] = 1
        elif prob2 == maximum:
            prediction[index] = 2
        elif prob3 == maximum:
            prediction[index] = 3
        else:
            prediction[index] = 4
        
        index += 1
    
    return prediction


        

        

## Testing Naive Bayes

In [19]:
test_prediction = naive_bayes_predict(X_test, probabilities, Y_train)
train_prediction = naive_bayes_predict(X_train, probabilities, Y_train)

In [20]:
def get_accuracy(prediction, labels):
    same_element = 0
    for i in range(len(prediction)):
        if prediction[i] == labels[i]:
            same_element += 1
    return same_element / len(prediction)


In [22]:
test_accuracy = get_accuracy(test_prediction, Y_test)
train_accuracy = get_accuracy(train_prediction, Y_train)

print("Train Accuracy:")
print(train_accuracy)
print("---------------------------")
print("Test Accuracy:")
print(test_accuracy)


Train Accuracy:
0.9890815671162492
---------------------------
Test Accuracy:
0.9505988023952096


# Part b)

## Gaussian Class Conditionals

In [23]:
def calculate_cov(data):
    # calculate the variance of the data given that the features are independent as the TA said
    var = np.var(data, axis=0)
    # because TA said features are independent the covariance matrix is diagonal
    cov = np.zeros((len(var), len(var)))
    index = 0

    for i in range(len(var)):
        # if variance is 0 then add a small bias term
        if(var[i] == 0):
            cov[index][index] = 1e-7
        else:
            #if variance is non zero assign the variance to the diagonal
            cov[index][index] = var[i]
        index += 1

    return cov

In [24]:
def trainGCC(data, labels):
    # TODO: Complete function
    labels = labels.flatten()
    data_class0 = data[labels == 0]
    data_class1 = data[labels == 1]
    data_class2 = data[labels == 2]
    data_class3 = data[labels == 3]
    data_class4 = data[labels == 4]


    # Calculate the covariance matrix for each class
    cov0 = calculate_cov(data_class0)
    cov1 = calculate_cov(data_class1)
    cov2 = calculate_cov(data_class2)
    cov3 = calculate_cov(data_class3)
    cov4 = calculate_cov(data_class4)

        # Calculate the means for each class
    mean0 = np.mean(data_class0, axis=0)
    mean1 = np.mean(data_class1, axis=0)
    mean2 = np.mean(data_class2, axis=0)
    mean3 = np.mean(data_class3, axis=0)
    mean4 = np.mean(data_class4, axis=0)

    return (mean0, cov0, mean1, cov1, mean2, cov2,
             mean3, cov3, mean4, cov4)

In [27]:
def calculate_inverses(cov0, cov1, cov2, cov3, cov4):
    inv_cov0 = np.zeros(cov0.shape)
    inv_cov1 = np.zeros(cov1.shape)
    inv_cov2 = np.zeros(cov2.shape)
    inv_cov3 = np.zeros(cov3.shape)
    inv_cov4 = np.zeros(cov4.shape)

    # because the covariance matrices are diagonal the inverse is just the reciprocol of every element on the diagonal

    np.fill_diagonal(inv_cov0, np.reciprocal(np.diag(cov0)))
    np.fill_diagonal(inv_cov1, np.reciprocal(np.diag(cov1)))
    np.fill_diagonal(inv_cov2, np.reciprocal(np.diag(cov2)))
    np.fill_diagonal(inv_cov3, np.reciprocal(np.diag(cov3)))
    np.fill_diagonal(inv_cov4, np.reciprocal(np.diag(cov4)))

    return inv_cov0, inv_cov1, inv_cov2, inv_cov3, inv_cov4


In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X_frequency, labels, test_size=0.30, train_size=0.70, random_state=20)

(mean0, cov0, mean1, cov1, mean2, cov2, mean3, cov3, mean4, cov4) = trainGCC(X_train, Y_train)

inv_cov0, inv_cov1, inv_cov2, inv_cov3, inv_cov4 = calculate_inverses(cov0, cov1, cov2, cov3, cov4)


In [29]:
def compute_log_gaussian(x, mean, cov, cov_inv):
    # Note we can remove the 1/2pi ^d cuz that is a common scalar for all of them given that they all have same features
    new_x = x.reshape(len(x), 1)
    mean = mean.reshape(len(mean), 1)

    # Because the covariance is diagonal, the determinant is the product of the diagonal
    # Taking the log of that just leads to the sum of the log diagonals by log rules
    cov_abs = np.sum(np.log(np.diag(cov)))
    matrix1 = -0.5 * np.matmul(np.matmul((new_x-mean).T, cov_inv), (new_x-mean))

    return  (-0.5) * cov_abs + matrix1


In [30]:
def predictGGC(X_test, Y_train, mean0, cov0, mean1, cov1, mean2, cov2, mean3, cov3, mean4, cov4):
    
    #get the initial class probabilities and log them since we are computing the log gaussian
    init_prob0, init_prob1, init_prob2, init_prob3, init_prob4 = get_class_probabilities(Y_train)
    init_prob0, init_prob1, init_prob2, init_prob3, init_prob4 = np.log(init_prob0), np.log(init_prob1), np.log(init_prob2), np.log(init_prob3), np.log(init_prob4) 

    prediction = np.empty((X_test.shape[0]))
    index = 0
    for row in X_test:
        # find the gaussian for each class given the row
        class0 = compute_log_gaussian(row, mean0, cov0, inv_cov0) + init_prob0
        class1 = compute_log_gaussian(row, mean1, cov1, inv_cov1) + init_prob1
        class2 = compute_log_gaussian(row, mean2, cov2, inv_cov2) + init_prob2
        class3 = compute_log_gaussian(row, mean3, cov3, inv_cov3) + init_prob3
        class4 = compute_log_gaussian(row, mean4, cov4, inv_cov4) + init_prob4

        # The classification depends on which gaussian value is the largest
        minimum = max(class0, class1, class2, class3, class4)

        if class0 == minimum:
            prediction[index] = 0
        elif class1 == minimum:
            prediction[index] = 1
        elif class2 == minimum:
            prediction[index] = 2
        elif class3 == minimum:
            prediction[index] = 3
        else:
            prediction[index] = 4

        index += 1
    
    return prediction

        

    
    



    
    
    
    


## Predict Both the test data and the train data

In [32]:
prediction_test = predictGGC(X_test, Y_train, mean0, cov0, mean1, cov1, mean2, cov2, mean3, cov3, mean4, cov4)
prediction_train = predictGGC(X_train, Y_train, mean0, cov0, mean1, cov1, mean2, cov2, mean3, cov3, mean4, cov4)

In [33]:
accuracy = get_accuracy(prediction_test, Y_test)
print("Test Accuracy:")
print(accuracy)
print("---------------------------")
accuracy_train = get_accuracy(prediction_train, Y_train)
print("Train Accuracy:")
print(accuracy_train)

Test Accuracy:
0.9131736526946108
---------------------------
Train Accuracy:
1.0
